In [ ]:
# default_exp pynamodb

# pynamodb
> special pynamodb attributes

## Imports

In [ ]:
#export
from pynamodb.attributes import Attribute, UnicodeAttribute, NumberAttribute
from typing import Any, Optional, Type, TypeVar
from enum import Enum
import requests, dpath.util, yaml, jsonschema, json, os, pynamodb, pytest
from pynamodb.models import Model
from datetime import datetime, timezone
from awsSchema.apigateway import Event, Response
from jsonschema import ValidationError
from typing import Optional
from nicHelper.schema import validateUrl
from beartype import beartype

## Error classes

In [ ]:
#export
class PynamoDBSavingError(Exception):
  pass
class PynamoDBSchemaValidationError(Exception):
  pass

# SchemaAttribute class
a class which automatically parse and check data against json schema

In [ ]:
#export
class SchemaAttribute(Attribute):
  attr_type = pynamodb.constants.STRING
  def __init__(self, schemaUrl:str, path:str = '/', isYaml=True, 
               headers={'Cache-Control': 'no-cache'}, 
               envName = 'SCHEMA_ATTRIBUTE', **kwargs: Any) -> None:
      """
      schemaUrl:str, 
      path:str = '/', 
      isYaml=True,  :yaml::Bool:: whether the schema is in yaml or json
      headers={'Cache-Control': 'no-cache'},
      :path::str:: the path of the object of interest in schema, if the schema is at root then '/'
      envName::str:: the name of schema to save to the environment
      """
      super().__init__(**kwargs)
      try:
        if isYaml: # yaml schema
          schema:dict = yaml.load(requests.get(schemaUrl, headers=headers).text, Loader = yaml.FullLoader)
        else: # probably json
          schema:dict = requests.get(schemaUrl, headers).json()
      except Exception as e:
        print(f'error parsing schema {e}')
        schema:dict = {}
          
      self.schema = dpath.util.get(schema, path) # get to the path in schema
      os.environ[envName] = json.dumps(self.schema)

  def deserialize(self, value: str) -> dict:
    return json.loads(value)

  def serialize(self, value:dict) -> str:
    res = jsonschema.validate(value,self.schema)
    return json.dumps(value)

# Supermodel
a class which add some functionalities on top of the standard pynamodb model, it sets id_ as the hash key and gives
* fromDict functions 
* repr as a dict



In [ ]:
#export
extraDocsString = '''
  def pullOutKeys(self):
    self.id_ = self.data['id']
  def update(self, inputDict:dict):
    self.data.update(inputDict)
  def fromDict(cls, inputDict:dict):
    return cls(data = inputDict)
  def toDict(self):
    return self.data
    
'''

In [ ]:
#export
class SuperModel(Model):
  f'''
  a model intended to use as a standard single json object saving
  please override functions including
  - fromDict
  - toDict
  - pullOutKeys
  - update
  override as necessary for example
  {extraDocsString}
  '''
  #id_ = UnicodeAttribute(hash_key=True)
  data = SchemaAttribute(schemaUrl='https://gist.githubusercontent.com/thanakijwanavit/e2720d091ae0cef710a49b57c0c9cd4c/raw/ed2d322eac4900ee0f95b431d0f9067a40f3e0f0/squirrelOpenApiV0.0.3.yaml', null=True)
  lastEdited = NumberAttribute()
  creationTime = NumberAttribute()
  
  def __repr__(self):
    return json.dumps(vars(self)['attribute_values'])
  
  ## query override
  @classmethod
  def queryId(cls, hash_key, **kwargs):
    '''
    just like query but make resul
    '''
    r = cls.query(hash_key, **kwargs)
    res = []
    for i in r:
      i.pullOutKeys()
      res.append(i)
    return iter(res)
      
  
  ## moving to and from dict
  @classmethod
  def fromDict(cls, inputDict:dict):
    '''
    turn dict into class
    note that this assume data as the only column, please override if required
    '''
    return cls(data = inputDict)
  
  def toDict(self):
    '''
    turn class into a dictionary
    note that this is set to return self.data by default, please override if needed
    '''
    return self.data
  
  #### saving ####
  def pullOutKeys(self):
    '''
    update the keys with data: please override this function by pulling out keys
    
    for example
    self.orderId = self.data['orderId']
    self.ownerId = self.data['ownerId']
    self.basketId = self.data['basketId']
    '''
    print('please dont foreget to override the pullOutKeys function if needed')
    self.id_ = self.data['id']
    
  def recordTime(self, tz=timezone.utc):
    '''record last edited and creation time'''
    self.lastEdited = datetime.now(tz=tz).timestamp() # record last edited
    if not self.creationTime: # record creation time
      self.creationTime = datetime.now(tz=tz).timestamp()
    
  def save(self):
    ''' 
    please override pullOutKeys function
    see docs
    this function performs following before saving the record
      1. self.recordTime()
      2. self.pullOutKeys()
    '''
    self.recordTime()
    self.pullOutKeys()
      
    try: 
      super().save()
      return self
    except ValidationError as e:
      raise PynamoDBSchemaValidationError(f'failed validation \n {e}')
    except Exception as e:
      raise PynamoDBSavingError(f'error saving id  {e}')
  def update(self, inputDict:dict):
    '''
    update with dictionary input
    please override as necessary
    '''
    self.data.update(inputDict)
  

### Test

In [ ]:
## check schema
testSchema = 'https://gist.githubusercontent.com/thanakijwanavit/e2720d091ae0cef710a49b57c0c9cd4c/raw/ed2d322eac4900ee0f95b431d0f9067a40f3e0f0/squirrelOpenApiV0.0.3.yaml'
schema = yaml.load(requests.get(testSchema).text, Loader= yaml.Loader)
path = 'components/schemas/Location'
dpath.util.get(schema, path)

{'type': 'object',
 'required': ['id',
  'type',
  'street_address',
  'city',
  'state',
  'zip',
  'capacity',
  'status'],
 'properties': {'id': {'type': 'string', 'format': 'uuid'},
  'type': {'type': 'string', 'enum': ['pick up', 'drop off', 'overnight']},
  'street_address': {'type': 'string'},
  'city': {'type': 'string'},
  'state': {'type': 'string',
   'pattern': '^(?:(A[KLRZ]|C[AOT]|D[CE]|FL|GA|HI|I[ADLN]|K[SY]|LA|M[ADEINOST]|N[CDEHJMVY]|O[HKR]|PA|RI|S[CD]|T[NX]|UT|V[AT]|W[AIVY]))$'},
  'zip': {'type': 'string', 'pattern': '(^\\d{5}$)|(^\\d{5}-\\d{4}$)'},
  'status': {'type': 'string', 'enum': ['open', 'in use']},
  'created': {'type': 'string', 'format': 'date-time'},
  'modified': {'type': 'string', 'format': 'date-time'}}}

### TestModel

In [ ]:
schemaUrl = 'https://raw.githubusercontent.com/thanakijwanavit/villaMasterSchema/master/Product.json'
from typing import Any
class TestModel(SuperModel):
  class Meta:
    table_name="colab-test-sensitive-column"
    region = 'ap-southeast-1'
  data = SchemaAttribute(schemaUrl = schemaUrl, null=True)
  phoneHash = UnicodeAttribute(hash_key=True)
  
    
  # Overrides
  def pullOutKeys(self)->None:
    self.phoneHash = str(self.data.get('phoneHash') or self.data.get('iprcode') or self.data.get('id') )

  @beartype
  def toDict(self)->dict:
    return self.data
    
  @classmethod
  @beartype
  def fromDict(cls, inputDict:dict)->Any:
    return cls(data=inputDict)
    
  @beartype  
  def update(self,inputDict:dict)->None:
    self.data.update(inputDict)

In [ ]:
d = TestModel('123', data={'iprcode': 4, 'cprcode': 123 , 'oprCode': '123', 'orderId': 123})
assert d.pullOutKeys() == None
assert type(d.toDict()) == dict
assert d.update({'cprcode': 234}) ==None

### success

In [ ]:
from nicHelper.exception import errorString
try:
  test = TestModel.fromDict({'iprcode': 4, 'cprcode': 123 , 'oprCode': '123', 'orderId': 123})
  test.save()
except Exception as e:
  print(e)
  print(errorString())


next(TestModel.query('1'))

{"data": {"type": "pick up", "street_address": "123", "id": "123", "city": "sth", "state": "CA", "zip": "23523", "capacity": 5, "status": "open"}}

In [ ]:

next(TestModel.query('1'))

{"data": {"type": "pick up", "street_address": "123", "id": "123", "city": "sth", "state": "CA", "zip": "23523", "capacity": 5, "status": "open"}}

### fail

In [ ]:
try:
  TestModel(
    data = {'iprcode': '4', 'cprcode': 123 , 'oprCode': '123'}
  ).save()
except Exception as e:
  print(e)



next(TestModel.query('1'))

failed validation 
 '4' is not of type 'integer'

Failed validating 'type' in schema['properties']['iprcode']:
    {'type': 'integer'}

On instance['iprcode']:
    '4'


{"data": {"type": "pick up", "street_address": "123", "id": "123", "city": "sth", "state": "CA", "zip": "23523", "capacity": 5, "status": "open"}}

### nested

In [ ]:
schemaUrl = 'https://gist.githubusercontent.com/thanakijwanavit/e2720d091ae0cef710a49b57c0c9cd4c/raw/ed2d322eac4900ee0f95b431d0f9067a40f3e0f0/squirrelOpenApiV0.0.3.yaml'
path = '/components/schemas/Location'
class ProductModel(SuperModel):
  class Meta:
    table_name="colab-test-sensitive-column"
    region = 'ap-southeast-1'
  phoneHash = UnicodeAttribute(hash_key=True)
  data = SchemaAttribute(schemaUrl = schemaUrl,path=path, null=True)
  
  def pullOutKeys(self):
    self.phoneHash = self.data.get('id')

  
def test_nested():
  result = {}
  try:
    ProductModel(
      data = {'type': 'something invalid', 'street_address': '123' }
    ).save()
  except Exception as e:
    print('faulty data is rejected')
    result['errorModel'] = True

  try:
    data = {'type': 'pick up', 'street_address': '123' , 'id': '123', 'city':'sth', 'state': 'CA', 'zip':'23523', 'capacity':5, 'status':'open'}
    product:ProductModel = ProductModel.fromDict(data)
    result['successModel'] = True
  except Exception as e:
    print(f'valid data is rejected\n{e}')
    result['successModel'] = False
  

  assert next(TestModel.query('1')).data == {'type': 'pick up', 'street_address': '123' , 'id': '123', 'city':'sth', 'state': 'CA', 'zip':'23523', 'capacity':5, 'status':'open'}
  assert result['successModel'] == True, 'success model didnt save properly'
  assert result['errorModel'] == True, 'error model went through'
  
test_nested()

faulty data is rejected


# Standard functions

## Create data

In [ ]:
#export
def createData(event:dict, hashKeyName: str,mainClass:Model, schemaUrl:Optional[str] = None ,schemaFormat:str ='yaml', *args):
  '''
    create a new row of data
  '''
  # parse output
  query:dict = Event.parseBody(event) 
  
  # check schema if provided
  if schemaUrl: 
    try: validateUrl(schemaUrl, query, format_ = schemaFormat)
    except ValidationError as e: return Response.returnSuccess(f'{e}')
  
  # check for key
  if hashKeyName not in query:  
    return Response.returnError(message=f'missing {hashKeyName}') 
  
  # check if object exist 
  if next(mainClass.query(query[hashKeyName]),None): 
    return Response.returnError(message=f'item with the same hash key exists')
  
  # make pynamodb object
  item:mainClass = mainClass.fromDict(query)
    
  # try to save
  try: 
    item.save()
    return Response.returnSuccess(body=item.toDict())
  
  except ValidationError as e: # error validation handle
    return Response.returnError(f'validation error \n {e}')
  
  except Exception as e: # error handle
    return Response.returnError(f'unknown error \n {e} \n errorString())')

### Usage example

In [ ]:
## lambda create function
def create (event, *args):
  body = Event.parseBody(event)
  body['id'] = body['phoneHash']
  
  event2 = Event.getInput(body)
  r = createData(event2, hashKeyName='phoneHash', mainClass=TestModel)
  if r.get('statusCode') != 200: return r
  r2 = next(TestModel.query(body['phoneHash']), None)
  if not r2: return Response.returnError('st wrong with saving, saving but didnt go through')
  return Response.returnSuccess(r2)

### Test

#### Success

In [ ]:
schemaUrl = 'https://raw.githubusercontent.com/thanakijwanavit/villaMasterSchema/master/Product.json'
data = {'phoneHash': '123','iprcode': 4, 'cprcode': 123 , 'oprCode': '123'}
event = Event.getInput(data)
item = next(TestModel.queryId('123'), None)
print('existing item is :',item)
# delete item if exist
if item:
  print(item.delete())
create(event)

existing item is : {"lastEdited": 1625508893.420898, "creationTime": 1625508893.330854, "data": {"phoneHash": "123", "iprcode": 5, "cprcode": 123, "oprCode": "1234", "id": "123"}, "phoneHash": "123"}
{'ConsumedCapacity': {'CapacityUnits': 1.0, 'TableName': 'colab-test-sensitive-column'}}


{'body': '{"phoneHash":"123","iprcode":4,"cprcode":123,"oprCode":"123","id":"123"}',
 'statusCode': 200,
 'headers': {'Access-Control-Allow-Headers': '*',
  'Access-Control-Allow-Origin': '*',
  'Access-Control-Allow-Methods': '*'}}

In [ ]:
createData(Event.getInput(data), hashKeyName='phoneHash', mainClass =TestModel)
next(TestModel.query('123'))

{"lastEdited": 1626617229.393203, "creationTime": 1626617229.393216, "data": {"phoneHash": "123", "iprcode": 4, "cprcode": 123, "oprCode": "123", "id": "123"}}

In [ ]:
test = TestModel(data = data)
test.save()
test.phoneHash
next(TestModel.query('123'))

{"lastEdited": 1626617229.432975, "creationTime": 1626617229.432986, "data": {"phoneHash": "123", "iprcode": 4, "cprcode": 123, "oprCode": "123"}}

## GetData

In [ ]:
#export
from awsSchema.apigateway import Event, Response
from jsonschema import ValidationError
from typing import Optional
from nicHelper.schema import validateUrl

def getData(hashKeyName:str, mainClass: Model):
  '''
    create a new basket
  '''
  # get data
  try:
    r:Optional[Model] = next(mainClass.query(hashKeyName), None)
  except Exception as e:
    return Response.returnError(f'failed to query with error {e}')
  
  # product not found
  if not r: return Response.returnError(f'not found')
  #success
  else: return Response.returnSuccess(r.toDict())

### usage

In [ ]:
def lambdaGet(event, *args):
  query = Event.parseBody(event)
  if 'key' not in query: return Response.returnError(f'missing key')
  return getData(query['key'], TestModel)

### Test

#### Success

In [ ]:
data = {'phoneHash': '123','iprcode': 4, 'cprcode': 123 , 'oprCode': '123'}
event = Event.getInput(data)
create(event)

lambdaGet(Event.getInput({'key': '123'}))

{'body': '{"phoneHash":"123","iprcode":4,"cprcode":123,"oprCode":"123"}',
 'statusCode': 200,
 'headers': {'Access-Control-Allow-Headers': '*',
  'Access-Control-Allow-Origin': '*',
  'Access-Control-Allow-Methods': '*'}}

#### failed wrong key type

In [ ]:
lambdaGet(Event.getInput({'key': 123}))

{'body': '{"error":"failed to query with error object of type \'int\' has no len()"}',
 'statusCode': 400,
 'headers': {'Access-Control-Allow-Headers': '*',
  'Access-Control-Allow-Origin': '*',
  'Access-Control-Allow-Methods': '*'}}

## update data

In [ ]:
# export
def updateData(event:dict, hashKeyName: str,mainClass:Model, 
               schemaUrl:Optional[str] = None ,schemaFormat:str ='yaml', *args):
  '''
  updating data based on the new input
  
  event:dict: object gathered from apiGatewayProxy
  hashKeyName:str: the name of the hash key
  mainClass:Model: pynamodb model class for this object
  schemaUrl:Optional[str]: url of the input schema for validation
  schemaFormat:Enum['yaml', 'json']
  '''
  # parse output
  query:dict = Event.parseBody(event) 

  # check schema if provided
  if schemaUrl: 
    try: validateUrl(schemaUrl, query,format_ = schemaFormat)
    except ValidationError as e: return Response.returnSuccess(f'{e}')

  # check for key
  if hashKeyName not in query:  
    return Response.returnError(message=f'missing {hashKeyName}') 

  # check if object exist 
  item:mainClass = next(mainClass.query(query[hashKeyName]),None)
  if not item: 
    return Response.returnError(message=f'item with the same hash key doesnt exist')

  # update the data
  item.update(query)

  # try to save
  try: 
    item.save()
    return Response.returnSuccess(body=item.toDict())

  except ValidationError as e: # error validation handle
    return Response.returnError(f'validation error \n {e}')

  except Exception as e: # error handle
    return Response.returnError(f'unknown error \n {e} \n errorString())')
    

### sample Usage

In [ ]:
def update(event, *args):
  body = Event.parseBody(event)
  body['id'] = body['phoneHash']
  
  event2 = Event.getInput(body)
  hashKeyname = 'id'
  return updateData(event2, hashKeyName=hashKeyname, mainClass=TestModel)

In [ ]:
r = create(Event.getInput({'phoneHash': '123','iprcode': 5, 'cprcode': 123 , 'oprCode': '123'}))
r = update(Event.getInput({'phoneHash': '123','iprcode': 5, 'cprcode': 123 , 'oprCode': '1234'}))
lambdaGet(Event.getInput({'key':'123'}))

{'body': '{"phoneHash":"123","iprcode":5,"cprcode":123,"oprCode":"1234","id":"123"}',
 'statusCode': 200,
 'headers': {'Access-Control-Allow-Headers': '*',
  'Access-Control-Allow-Origin': '*',
  'Access-Control-Allow-Methods': '*'}}